<a href="https://colab.research.google.com/github/MADY-777/Traffic-Sign-Recognition-System./blob/main/weather.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

section 1 : importing librariies

In [2]:
import requests
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.metrics import mean_squared_error
from datetime import datetime
import pytz

In [3]:
api_key='44e960f8a0134a935c8c54e2b5399e8f'
base_url='https://api.openweathermap.org/data/2.5/'

fetch current weather

In [4]:
def get_current_weather(city):
    url=f"{base_url}weather?q={city}&appid={api_key}&units=metric"
    response=requests.get(url)
    data= response.json()

    if response.status_code == 200:
        return{
            'city':data['name'],
            'current_temp':round(data['main']['temp']),
            'humidity':round(data['main']['humidity']),
            'feels_like':round(data['main']['feels_like']),
            'temp_min':round(data['main']['temp_min']),
            'temp_max':round(data['main']['temp_max']),
            'description':data['weather'][0]['description'],
            'country':data['sys']['country'],
            'preasure':data['main']['pressure'],
            'Wind_Gust_Speed':data['wind']['speed'],
        }
    else:
        print(f"Error fetching weather for {city}: {data.get('message', 'Unknown error')}")
        return None

read historical data

In [5]:
def read_historical_data(filename):
    df=pd.read_csv(filename)
    df=df.dropna()
    df=df.drop_duplicates()
    return df

3.prepare data for training

In [6]:
def prepare_data(data):
   le=LabelEncoder()
   data['WindGustDir']=le.fit_transform(data['WindGustDir'])
   data['RainTomorrow']=le.fit_transform(data['RainTomorrow'])
   x=data[['MinTemp','MaxTemp','WindGustSpeed','Humidity','Pressure','Temp']]
   y=data['RainTomorrow']
   return x,y,le

4.Training rain prediction model

In [7]:
def train_rain_model(x,y):
  x_Train,x_test,y_Train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)
  model=RandomForestClassifier(n_estimators=100,random_state=42)
  model.fit(x_Train,y_Train)
  y_pred=model.predict(x_test)
  print("mean squared error for rain model")
  print(mean_squared_error(y_test,y_pred))
  return model

5.Prepare regression Data

In [8]:
def prepare_regression_data(data,feature):
  x,y=[],[]
  for i in range(len(data)-1):
    x.append(data[feature].iloc[i])
    y.append(data[feature].iloc[i+1])
  x=np.array(x).reshape(-1,1)
  y=np.array(y)
  return x,y



6.training regression data

In [9]:
def train_regression_model(x,y):
  model=RandomForestClassifier(n_estimators=100,random_state=42)
  model.fit(x,y)
  return model


7.Predict Future

In [10]:
def predict_future(model,current_value):
  predictions = [current_value]
  for i in range(5):
    next_value=model.predict(np.array(predictions[-1]))
    predictions.append(next_value[0])
  return precitions[1:]

8.weather analysis function

In [12]:
def read_historical_data(filename):
    df=pd.read_csv(filename)
    df=df.dropna()
    df=df.drop_duplicates()
    return df

def weather_view():
  # Redefine train_regression_model locally to use RandomForestRegressor
  def train_regression_model(x,y):
    model=RandomForestRegressor(n_estimators=100,random_state=42)
    model.fit(x,y)
    return model

  # Redefine predict_future locally to handle 2D array input
  def predict_future(model,current_value):
    predictions = [current_value]
    for i in range(5):
      next_value=model.predict(np.array(predictions[-1]).reshape(-1, 1))
      predictions.append(next_value[0])
    return predictions[1:]

  city=input("enter the city name = ")
  current_weather=get_current_weather(city)
  if current_weather is None:
      print("Could not fetch current weather. Exiting.")
      return

  historical_data=read_historical_data('/content/weather.csv')
  print("Columns in historical_data:", historical_data.columns.tolist())
  # Use a copy to avoid modifying original historical_data in prepaer_data
  x,y,le= prepare_data(historical_data.copy())

  # To get a LabelEncoder for WindGustDir without modifying prepaer_data:
  # 1. Load historical data again to access original 'WindGustDir' strings
  original_historical_data = pd.read_csv('/content/weather.csv')
  original_historical_data = original_historical_data.dropna()
  original_historical_data = original_historical_data.drop_duplicates()

  le_wind_dir_for_prediction = LabelEncoder()
  le_wind_dir_for_prediction.fit(original_historical_data['WindGustDir'])

  rain_model=train_rain_model(x,y)

  # Get a default encoded WindGustDir for current weather prediction
  default_wind_dir_str = 'NW' # Assuming 'NW' is a common direction in historical data
  try:
    encoded_default_wind_dir = le_wind_dir_for_prediction.transform([default_wind_dir_str])[0]
  except ValueError:
    print(f"Warning: '{default_wind_dir_str}' not in WindGustDir categories. Assigning 0 as fallback.")
    encoded_default_wind_dir = 0 # Fallback to 0 if 'NW' is not in categories

  current_data={
      'MinTemp':current_weather['temp_min'],
      'MaxTemp':current_weather['temp_max'],
      'WindGustDir': encoded_default_wind_dir, # Added WindGustDir, encoded
      'WindGustSpeed':current_weather['Wind_Gust_Speed'],
      'Humidity':current_weather['humidity'], # Corrected key to 'Humidity' (uppercase)
      'Pressure':current_weather['preasure'],
      'Temp':current_weather['current_temp']
  }
  current_df = pd.DataFrame([current_data])

  # Ensure the order of columns in current_df matches x.columns
  current_df = current_df[x.columns] # Reorder columns to match training features

  rain_prediction=rain_model.predict(current_df)[0]
  x_temp,y_temp=prepare_regression_data(historical_data,'Temp')
  x_humidity,y_humidity=prepare_regression_data(historical_data,'Humidity')
  temp_model=train_regression_model(x_temp,y_temp)
  humidity_model=train_regression_model(x_humidity,y_humidity)
  future_temp=predict_future(temp_model,current_weather['current_temp'])
  future_humidity=predict_future(humidity_model,current_weather['humidity'])

  # Import timedelta locally to fix AttributeError
  from datetime import timedelta
  timezone=pytz.timezone('Asia/Kolkata')
  current_time=datetime.now(timezone)
  next_hour=current_time+timedelta(hours=1)
  next_hour = next_hour.replace(minute=0,second=0)
  future_times=[next_hour+timedelta(hours=i) for i in range(5)]

  print(f"city:{city}{current_weather['country']}")
  print(f"current_temprature:{current_weather['current_temp']}")
  print(f"humidity:{current_weather['humidity']}")
  print(f"preasure:{current_weather['preasure']}")
  print(f"description:{current_weather['description']}")
  print(f"wind ust speed:{current_weather['Wind_Gust_Speed']}")
  print(f"Feels_like:{current_weather['feels_like']}")
  print(f"minimum temprature:{current_weather['temp_min']}c")
  print(f"maximum temprature:{current_weather['temp_max']}c")
  print(f"rain prediction:{'yes' if rain_prediction == 1 else 'no'}")
  print("future temprature prediction:")
  for time,temp in zip(future_times,future_temp):
    print(f"{time.strftime('%H:00')}:{round(temp,1)}c")
  print("future humidity prediction:")
  for time,humidity in zip(future_times,future_humidity):
    print(f"{time.strftime('%H:00')}:{round(humidity,1)}%")

weather_view()

enter the city name = london
Columns in historical_data: ['MinTemp', 'MaxTemp', 'WindGustDir', 'WindGustSpeed', 'Humidity', 'Pressure', 'Temp', 'RainTomorrow']
mean squared error for rain model
0.1506849315068493
city:londonGB
current_temprature:13
humidity:70
preasure:1015
description:light rain
wind ust speed:5.66
Feels_like:12
minimum temprature:12c
maximum temprature:13c
rain prediction:no
future temprature prediction:
20:00:13.5c
21:00:11.7c
22:00:11.6c
23:00:11.7c
00:00:11.6c
future humidity prediction:
20:00:64.8%
21:00:56.2%
22:00:45.2%
23:00:46.3%
00:00:48.2%
